## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
from math import ceil

In [2]:
effect_size = sms.proportion_effectsize(0.2, 0.19)

required_n = sms.NormalIndPower().solve_power(
    effect_size,
    power=0.8,
    alpha=0.05,
    ratio=1
    )

required_n = ceil(required_n)

print(f"Для досягнення відчутного ефекту при тестуванні гіпотез,")
print(f"треба зібрати дані про {required_n} користувачів")

Для досягнення відчутного ефекту при тестуванні гіпотез,
треба зібрати дані про 24638 користувачів


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [3]:
data_path = '../data/statistical_hypothesis/cookie_cats.csv'
df = pd.read_csv(data_path)

In [4]:
retention_7_mean = df.groupby("version")["retention_7"].mean()

retention_7_mean

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64

Статистична гіпотеза

H₀ (нульова):
Retention_7 у версіях gate_30 та gate_40 однаковий.

H₁ (альтернативна):
Retention_7 у версії gate_30 вищий, ніж у gate_40.

3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [5]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [6]:
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [7]:
results = df.groupby("version")["retention_7"].agg(["sum", "count"])

successes = results['sum'].values
nobs = results['count'].values

In [8]:
successes, nobs

(array([8502, 8279]), array([44700, 45489]))

In [9]:
z_stat, pval = proportions_ztest(successes, nobs=nobs, alternative='larger')
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f"z statistic: {z_stat:.4f}")
print(f"p-value: {pval:.4f}")
print(f"Довірчий інтервал 95% для групи control: [{lower_con:.4f}, {upper_con:.4f}]")
print(f"Довірчий інтервал 95% для групи treatment: [{lower_treat:.4f}, {upper_treat:.4f}]")

z statistic: 3.1644
p-value: 0.0008
Довірчий інтервал 95% для групи control: [0.1866, 0.1938]
Довірчий інтервал 95% для групи treatment: [0.1785, 0.1855]


1. На рівні значущості 0.05 є статистично значущі докази, що версія гри з воротами на 30 рівні (gate_30) має вищий показник утримання на 7-й день, ніж версія з воротами на 40 рівні (gate_40).
2. Довірчі інтервали не перекриваються. Уся оцінка для gate_30 лежить вище, ніж для gate_40. Це додатково підтверджує результат z-тесту.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


# Формулювання гіпотез для χ²-тесту

Ми перевіряємо, чи пов’язані між собою:
версія гри (gate_30, gate_40)
утримання на 7-й день (retention_7)

Нульова гіпотеза: Між версією гри та утриманням на 7-й день немає залежності
(змінні незалежні).

Альтернативна гіпотеза: Між версією гри та утриманням на 7-й день є статистична залежність
(змінні не є незалежними).

In [14]:
crosstab = pd.crosstab(df['version'], df['retention_7'])

In [18]:
chi2, p, dof, expected = stats.chi2_contingency(crosstab)

print(f"χ² = {chi2:.3f}")
print(f"p-value = {p:.5f}")
print(f"Ступені свободи = {dof}")
print("Очікувані частоти:\n", expected)

if p < 0.05:
    print("\n" + "Відхиляємо Н0. Між версією гри та утриманням на 7-й день є статистична залежність")
else:
    print("\n" + "Не відхиляємо Н0. Між версією гри та утриманням на 7-й день нема статистичної залежності/")

χ² = 9.959
p-value = 0.00160
Ступені свободи = 1
Очікувані частоти:
 [[36382.90257127  8317.09742873]
 [37025.09742873  8463.90257127]]

Відхиляємо Н0. Між версією гри та утриманням на 7-й день є статистична залежність
